# Определение токсичности комментариев с помощью BERT

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

## Установка библиотек

In [ ]:
!pip install transformers sentencepiece

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig, AutoModel
from transformers import AdamW, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == 'cpu':
    print('cpu')
else:
    n_gpu = torch.cuda.device_count()
    print(torch.cuda.get_device_name(0), n_gpu)

## Загрузка данных


In [ ]:
import pandas as pd
#train_df = pd.DataFrame.from_records(__flatten(train_data))
#test_df = pd.DataFrame.from_records(__flatten(test_data))
train_df = pd.read_csv("/content/drive/MyDrive/Программирование/м4 семестр/Диссертация/rutoxic/labeled.csv", encoding="utf8")
df = train_df
#df = pd.read_csv("/content/drive/MyDrive/Программирование/м4 семестр/Диссертация/jigsaw/jigsaw-toxic-comment-train-google-ru.csv", encoding="utf8")

TOXITY_TYPE = "toxic"


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Программирование/м4 семестр/Диссертация/jigsaw/jigsaw-toxic-comment-train-google-ru.csv", encoding="utf8")

df = df[(df[TOXITY_TYPE] == "1") | (df[TOXITY_TYPE] == "0") | (df[TOXITY_TYPE] == 1) | (df[TOXITY_TYPE] == 0)]
df.toxic = df[TOXITY_TYPE].astype(int)

tox_df = df[df[TOXITY_TYPE] == 1]
nottox_df = df[df[TOXITY_TYPE] == 0]

tox_l = len(tox_df)
nottox_l = len(nottox_df)


df = pd.concat([tox_df, nottox_df[:tox_l]])
test_df = df


In [ ]:
'''
from sklearn.model_selection import train_test_split

train_df, test_df  = train_test_split(df, test_size=0.2)

#train_df = df
'''

In [ ]:
'''
ins_df = test_df[test_df.insult == 1]
notins_df = test_df[test_df.insult == 0]

from sklearn.model_selection import train_test_split

train_ins_df, test_ins_df  = train_test_split(ins_df, test_size=0.3)
train_notins_df, test_notins_df = train_test_split(notins_df, test_size=0.3)

ins_l = len(train_ins_df)
notins_l = len(train_notins_df)

concat_dfs = [train_notins_df] + [train_ins_df for _ in range(notins_l // ins_l)]

train_df = pd.concat(concat_dfs)

test_df = pd.concat([test_ins_df, test_notins_df])
'''

In [ ]:
"""
ins_df = train_df[train_df.insult == 1]
notins_df = train_df[train_df.insult == 0]

ins_l = len(ins_df)
notins_l = len(notins_df)

concat_dfs = [notins_df[:ins_l], ins_df]

train_df = pd.concat(concat_dfs)
"""

In [ ]:
'''
import re

mat_re = r"\b((у|[нз]а|(хитро|не)?вз?[ыьъ]|с[ьъ]|(и|ра)[зс]ъ?|(о[тб]|под)[ьъ]?|(.\B)+?[оаеи])?-?([её]б(?!о[рй])|и[пб][ае][тц]).*?|(н[иеа]|[дп]о|ра[зс]|з?а|с(ме)?|о(т|дно)?|апч)?-?ху([яйиеёю]|ли(?!ган)).*?|(в[зы]|(три|два|четыре)жды|(н|сук)а)?-?бл(я(?!(х|ш[кн]|мб)[ауеыио]).*?|[еэ][дт]ь?)|(ра[сз]|[зн]а|[со]|вы?|п(р[ои]|од)|и[зс]ъ?|[ао]т)?п[иеё]зд.*?|(за)?п[ие]д[аое]?р((ас)?(и(ли)?[нщктл]ь?)?|(о(ч[еи])?)?к|юг)[ауеы]?|манд([ауеы]|ой|[ао]вошь?(е?к[ауе])?|юк(ов|[ауи])?)|муд([аио].*?|е?н([ьюия]|ей))|мля([тд]ь)?|лять|([нз]а|по)х|м[ао]л[ао]фь[яию])\b"

def clean_texts(texts):
    return list(map(lambda x: re.sub(mat_re, "<мат>", re.sub(r"[^\w\s\d]", ' ', x.replace("ё", "е"))), texts))
'''
def clean_texts(texts):
    return texts


In [ ]:
train_sentences = clean_texts(train_df["comment_text"].values)

#print(train_sentences[0])
train_sentences = [str(sentence) for sentence in train_sentences]
train_gt = [[int(l)] for l in train_df[TOXITY_TYPE].values]


test_sentences = clean_texts(test_df["comment_text"].values)

test_sentences = [ str(sentence) for sentence in test_sentences]
test_gt = [[int(l)] for l in test_df[TOXITY_TYPE].values]


## Inputs

In [ ]:
from transformers import AutoTokenizer, BertConfig


tokenizer = AutoTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=True) #bert-base-multilingual-cased sberbank-ai/sbert_large_nlu_ru sismetanin/rubert-toxic-pikabu-2ch DeepPavlov/rubert-base-cased

tokenized_texts = [["[CLS]"] + tokenizer.tokenize(sent)[:510]  + ["[SEP]"] for sent in train_sentences]
print (tokenized_texts[0])

In [ ]:
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
input_ids = pad_sequences(
    input_ids,
    maxlen=100,
    dtype="long",
    truncating="post",
    padding="post"
)
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]

In [ ]:
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(
    input_ids, train_gt, 
    random_state=42,
    test_size=0.1
)

train_masks, validation_masks, _, _ = train_test_split(
    attention_masks,
    input_ids,
    random_state=42,
    test_size=0.1
)

In [ ]:
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)

In [ ]:
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)

In [ ]:
train_labels

In [ ]:
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_dataloader = DataLoader(
    train_data,
    sampler=RandomSampler(train_data),
    batch_size=32
)

In [ ]:
validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_dataloader = DataLoader(
    validation_data,
    sampler=SequentialSampler(validation_data),
    batch_size=32
)

## Обучение модели

In [ ]:
from transformers import AdamW, AutoModelForSequenceClassification

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=2) #bert-base-multilingual-cased sismetanin/rubert-toxic-pikabu-2ch DeepPavlov/rubert-base-cased
model.cuda()

In [ ]:
warmup_prop=0.1

EPOCH_NUM = 2

optimizer = AdamW(model.parameters(), lr=2e-5)
'''
num_warmup_steps = int(warmup_prop * EPOCH_NUM * len(train_dataloader))
num_training_steps = EPOCH_NUM * len(train_dataloader)

scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps)

loss_fct = nn.BCEWithLogitsLoss(reduction='mean').to(device)
'''


In [ ]:
from IPython.display import clear_output

# Будем сохранять loss во время обучения
# и рисовать график в режиме реального времени
train_loss_set = []
train_loss = 0


# Обучение
# Переводим модель в training mode
model.train()


for i in range(EPOCH_NUM):
    print('=' * 50, f"EPOCH {i}", '=' * 50)
    for step, batch in enumerate(train_dataloader):
        # добавляем батч для вычисления на GPU
        batch = tuple(t.to(device) for t in batch)
        # Распаковываем данные из dataloader
        b_input_ids, b_input_mask, b_labels = batch
        
        # если не сделать .zero_grad(), градиенты будут накапливаться
        optimizer.zero_grad()
        
        # Forward pass
        loss = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)

        train_loss_set.append(loss[0].item())  
        
        # Backward pass
        loss[0].backward()
        
        # Обновляем параметры и делаем шаг используя посчитанные градиенты
        optimizer.step()

        # Обновляем loss
        train_loss += loss[0].item()
        
        # Рисуем график
        clear_output(True)
        plt.plot(train_loss_set)
        plt.title("Training loss")
        plt.xlabel("Batch")
        plt.ylabel("Loss")
        plt.show()
    
print("Loss на обучающей выборке: {0:.5f}".format(train_loss / len(train_dataloader)))


# Валидация
# Переводим модель в evaluation mode
model.eval()

valid_preds, valid_labels = [], []

for batch in validation_dataloader:   
    # добавляем батч для вычисления на GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Распаковываем данные из dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # При использовании .no_grad() модель не будет считать и хранить градиенты.
    # Это ускорит процесс предсказания меток для валидационных данных.
    with torch.no_grad():
        logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    # Перемещаем logits и метки классов на CPU для дальнейшей работы
    logits = logits[0].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    
    batch_preds = np.argmax(logits, axis=1)
    batch_labels = np.concatenate(label_ids)     
    valid_preds.extend(batch_preds)
    valid_labels.extend(batch_labels)



In [ ]:
print(classification_report(valid_labels, valid_preds))

# Оценка качества на отложенной выборке

In [ ]:
tokenized_texts = [["[CLS]"] + tokenizer.tokenize(sent)[:510]  + ["[SEP]"] for sent in test_sentences]
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

input_ids = pad_sequences(
    input_ids,
    maxlen=100,
    dtype="long",
    truncating="post",
    padding="post"
)

In [ ]:
attention_masks = [[float(i>0) for i in seq] for seq in input_ids]

prediction_inputs = torch.tensor(input_ids)
prediction_masks = torch.tensor(attention_masks)
prediction_labels = torch.tensor(test_gt)

prediction_data = TensorDataset(
    prediction_inputs,
    prediction_masks,
    prediction_labels
)

prediction_dataloader = DataLoader(
    prediction_data, 
    sampler=SequentialSampler(prediction_data),
    batch_size=32
)

In [ ]:
model.eval()
test_preds, test_labels = [], []

for batch in prediction_dataloader:
    # добавляем батч для вычисления на GPU
    batch = tuple(t.to(device) for t in batch)
    
    # Распаковываем данные из dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    # При использовании .no_grad() модель не будет считать и хранить градиенты.
    # Это ускорит процесс предсказания меток для тестовых данных.
    with torch.no_grad():
        logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)

    # Перемещаем logits и метки классов на CPU для дальнейшей работы
    logits = logits[0].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Сохраняем предсказанные классы и ground truth
    batch_preds = np.argmax(logits, axis=1)
    batch_labels = np.concatenate(label_ids)  
    test_preds.extend(batch_preds)
    test_labels.extend(batch_labels)

In [ ]:
print(classification_report(test_labels, test_preds))


In [ ]:
torch.cuda.empty_cache()

In [ ]:
copy_df = test_df.copy()
copy_df["predict"] = test_preds

copy_df.to_excel("test.xlsx")